In [34]:
import pandas as pd
from datetime import datetime

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Carga de los dataset necesarios

In [2]:
df_reviews = pd.read_csv('data/df_reviews_unido.csv')

In [3]:
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,items_count,price,genres,release_anio,developer,playtime_horas
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,277.0,19.99,Action,2009,Tripwire Interactive,5060.666667
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,1,888.0,NaN,Sin dato disponible,Sin dato disponible,Sin dato disponible,5539.916667
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Action,2013,"Hopoo Games, LLC",6180.266667
3,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Indie,2013,"Hopoo Games, LLC",6180.266667
4,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,RPG,2013,"Hopoo Games, LLC",6180.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130628,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Casual,2015,Trion Worlds,1862.433333
130629,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Free to Play,2015,Trion Worlds,1862.433333
130630,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Massively Multiplayer,2015,Trion Worlds,1862.433333
130631,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,RPG,2015,Trion Worlds,1862.433333


In [88]:
genre_ranking = pd.read_csv('data/genre_ranking_unido.csv')

In [89]:
genre_ranking

,genres,playtime_forever,ranking
0,Action,3113562606,1
1,Indie,1494622404,2
2,RPG,1041022718,3
3,Adventure,909995120,4
4,Simulation,867646306,5
5,Sin dato disponible,750732648,6
6,Strategy,659363841,7
7,Free to Play,610752945,8
8,Massively Multiplayer,446594080,9
9,Casual,252232854,10


In [4]:
utils.verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,user_url,[<class 'str'>],100.00,0.00,0
2,reviews_item_id,[<class 'int'>],100.00,0.00,0
3,reviews_helpful,[<class 'str'>],100.00,0.00,0
4,reviews_recommend,[<class 'bool'>],100.00,0.00,0
5,reviews_date,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,items_count,[<class 'float'>],89.18,10.82,14140
8,price,[<class 'float'>],93.44,6.56,8571
9,genres,[<class 'str'>],100.00,0.00,0


## userdata

In [23]:
def userdata(user_id):
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    cantidad_dinero = usuario['price'].sum()
    total_recomendaciones = usuario['reviews_recommend'].sum()
    total_items = usuario['items_count'].sum()
    
    total_reviews = len(df_reviews['user_id'].unique())
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': total_items.astype(int)
    }

In [25]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 254.88,
 'porcentaje_recomendacion': 0.09,
 'total_items': 529}

## countreviews

In [86]:
def countreviews(fecha_inicio, fecha_fin):
      
    user_data_entre_fechas = df_reviews[(df_reviews['reviews_date'] >= fecha_inicio) & (df_reviews['reviews_date'] <= fecha_fin)]
    total_usuarios = user_data_entre_fechas['user_id'].nunique()

    total_recomendacion = user_data_entre_fechas['reviews_recommend'].sum()
    porcentaje_recomendaciones = (total_usuarios / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }

In [87]:
countreviews('2011-11-05', '2012-12-24')

{'total_usuarios_reviews': 926, 'porcentaje_recomendaciones': 37.95}

### genre

In [115]:
def genre(genero):
    rank = genre_ranking[genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [119]:
genero = 'Simulation'
genre(genero)

{'rank': 5}

### userforgenre

In [125]:
def userforgenre(genero):
    data_por_genero = df_reviews[df_reviews['genres'] == genero]
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    top_users_dict = {}
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url'],
            #'playtime_horas': row['playtime_horas']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [127]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Steamified',
  'user_url': 'http://steamcommunity.com/id/Steamified'},
 2: {'user_id': 'idonothack',
  'user_url': 'http://steamcommunity.com/id/idonothack'},
 3: {'user_id': 'Efreak', 'user_url': 'http://steamcommunity.com/id/Efreak'},
 4: {'user_id': 'BlackReaperHei',
  'user_url': 'http://steamcommunity.com/id/BlackReaperHei'},
 5: {'user_id': '76561198058230663',
  'user_url': 'http://steamcommunity.com/profiles/76561198058230663'}}

### developer